# Import Libraries

In [31]:
import csv
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from sqlalchemy.orm import Session, sessionmaker, declarative_base
from sqlalchemy import create_engine

import app.models as models
from app.database import DATABASE_URL, TEST_DATABASE_URL

In [4]:
from dotenv import load_dotenv

load_dotenv()

True

# Set Up Database
Set up the database connection and create tables using SQLAlchemy's `Base.metadata.create_all`.

In [72]:
def prepare_db(URL):
    engine = create_engine(URL)
    SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
    Base = declarative_base()
    Base.metadata.create_all(bind=engine)
    db: Session = SessionLocal()
    Base.metadata.reflect(bind=engine)
    return engine, Base, db

In [73]:
engine, Base, db = prepare_db(DATABASE_URL)

In [75]:
model_classes = [v for k, v in models.__dict__.items() if isinstance(v, type) and hasattr(v, '__table__')]
tables = {model.__name__: model.__table__.name for model in model_classes}


In [82]:
for model, table in tables.items():
    file_name = f"db_dump/{table}.csv"
    with open(file_name, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        # Write header
        writer.writerow([column.name for column in Base.metadata.tables[table].columns])
        
        # Write data
        for row in db.query(Base.metadata.tables[table]).all():
            writer.writerow(row)